In [1]:
import tensorflow as tf
import keras
import numpy as np
import h5py
import scipy
import pandas as pd
import matplotlib.pyplot as plt
from linearmodels import PanelOLS
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
data = pd.read_csv('C:\\Users\\Shadeh\\Pictures\\pyfolder\\train_1.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145063 entries, 0 to 145062
Columns: 551 entries, Page to 2016-12-31
dtypes: float64(550), object(1)
memory usage: 609.8+ MB


,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,3.0,1.0,1.0,7.0,4.0,4.0,6.0,3.0,4.0,17.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,32.0,10.0,26.0,27.0,16.0,11.0,17.0,19.0,10.0,11.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,48.0,9.0,25.0,13.0,3.0,11.0,27.0,13.0,36.0,10.0


In [3]:
key = pd.read_csv('C:\\Users\\Shadeh\\Pictures\\pyfolder\\key_1.csv')
key_split = key.copy()
key_split['date'] = key_split.Page.apply(lambda a: a[-10:])
key_split['Page'] = key_split.Page.apply(lambda a: a[:-11])
key_split.head()

,Page,Id,date
0,!vote_en.wikipedia.org_all-access_all-agents,bf4edcf969af,2017-01-01
1,!vote_en.wikipedia.org_all-access_all-agents,929ed2bf52b9,2017-01-02
2,!vote_en.wikipedia.org_all-access_all-agents,ff29d0f51d5c,2017-01-03
3,!vote_en.wikipedia.org_all-access_all-agents,e98873359be6,2017-01-04
4,!vote_en.wikipedia.org_all-access_all-agents,fa012434263a,2017-01-05


In [4]:
def smape_test(y_true, y_pred):
    denominator = (y_true+y_pred)/200.0
    diff = np.abs(y_true-y_pred)/denominator
    diff[denominator==0] = 0.0
    return np.mean(diff)

In [5]:
data0 = data.copy()
data0.iloc[:,1:].isnull().sum(axis=0).describe()
data0 = data0.fillna(0)
for idx in [0, 50000]:
    print(idx)
    print(data0.iloc[idx,0])
    

0
2NE1_zh.wikipedia.org_all-access_spider
50000
Suicide_Squad_(Film)_de.wikipedia.org_all-access_spider


In [6]:
data = data.fillna(0)
y_train_true = data[list(data.columns[-120:-60])]
y_validate_true = data[list(data.columns[-60:])]
data1 = data.copy()
data1['index1'] = data1.index
data1 = data1[['index1']+list(data1.columns)[1:-1]]
data1[data1.columns[1:]] = data1[data1.columns[1:]].apply(lambda x: np.log(x+1))

X_train = data1[data1.columns[1:-120]]
y_train = data1[list(data1.columns[-120:-60])]
X_validate = data1[list(data1.columns[61:-60])]
y_validate = data1[list(data1.columns[-60:])]
X_test = data1[list(data1.columns[121:])]

In [7]:
X_train_norm = (X_train - X_train.min())/(X_train.max()-X_train.min())
X_validate_norm = (X_validate - X_validate.min())/(X_validate.max()-X_validate.min())
X_test_norm = (X_test - X_test.min())/(X_test.max()-X_test.min())
X_train1 = X_train_norm.values.reshape(X_train.shape[0], 1, X_train.shape[1])
X_validate1 = X_validate_norm.values.reshape(X_validate.shape[0], 1, X_validate.shape[1])
y_train1 = y_train.values
y_validate1 = y_validate.values
X_test1 = X_test_norm.values.reshape(X_test.shape[0], 1, X_test.shape[1])

In [8]:
LSTM_model = Sequential()
LSTM_model.add(LSTM(256, input_shape = (1,X_train.shape[1])))
LSTM_model.add(Dropout(0.3))
LSTM_model.add(Dense(60))
LSTM_model.compile(loss = 'mean_absolute_error', optimizer = 'rmsprop')
epochs = 10
checkpointer = ModelCheckpoint(filepath='weights.best.from_scratch1.hdf5', verbose=1, save_best_only=True)
print('Start training...')

LSTM_model.fit(X_train1, y_train1, validation_data=(X_validate1, y_validate1), 
              epochs=epochs, callbacks=[checkpointer], verbose=1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Start training...
Instructions for updating:
Use tf.cast instead.
Train on 145063 samples, validate on 145063 samples
Epoch 1/10
145063/145063 [==============================] - 720s 5ms/step - loss: 0.6318 - val_loss: 0.6302

Epoch 00001: val_loss improved from inf to 0.63023, saving model to weights.best.from_scratch1.hdf5
Epoch 2/10
145063/145063 [==============================] - 127s 875us/step - loss: 0.5714 - val_loss: 0.6331

Epoch 00002: val_loss did not improve from 0.63023
Epoch 3/10
145063/145063 [==============================] - 129s 891us/step - loss: 0.5605 - val_loss: 0.8652

Epoch 00003: val_loss did not improve from 0.63023
Epoch 4/10
145063/145063 [==============================] - 133s 918us/step - loss: 0.5518 - val_loss: 0.5430

Epoch 00004: val_loss improved from 0.63023 to 0.54

In [9]:

LSTM_model.load_weights('weights.best.from_scratch1.hdf5') 
y_validate_pred = LSTM_model.predict(X_validate1)
y_validate_pred1 = pd.DataFrame(data = y_validate_pred, columns = list(y_validate.columns))
y_val_pred = y_validate_pred1.apply(lambda x: np.exp(x)-1)
for i in list(y_val_pred.columns):
    y_val_pred[i] = y_val_pred[i].apply(lambda x: round(x))

y_val_pred[y_val_pred<0]=0
print('SAMPE of validate set when training on X_train: ', smape_test(y_validate_true.stack(), y_val_pred.stack()))

SAMPE of validate set when training on X_train:  46.73116408514589


In [10]:
y_train_true1 = y_train.copy()
y_train_true1['V'] = y_train_true.median(axis=1)
y_train_true1.head()

for i in list(y_validate_true.columns):
    y_train_true1[i] = y_train_true1['V']

smape_test(y_validate_true.stack(), y_train_true1[y_train_true1.columns[-60:]].stack())

47.96375802375267

In [11]:
X_test_P = data[['Page']+list(data.columns[-7:])]
LSTM_model.load_weights('weights.best.from_scratch1.hdf5')
y_test_pred = LSTM_model.predict(X_test1)
out_cols = list(pd.unique(key_split['date'].values))
y_test_pred1 = pd.DataFrame(data = y_test_pred, columns = list(out_cols))
y_test_pred2 = y_test_pred1.apply(lambda x: np.exp(x)-1)
y_test_pred2[y_test_pred2<0] =0
test_out = pd.merge(X_test_P, y_test_pred2, left_index = True, right_index = True)
test_out1 = test_out[['Page']+list(test_out.columns.values[-60:])]
test_out_120D = pd.melt(test_out1, id_vars='Page', var_name='date', value_name='Visits_120D')
test_out_120D.head()
checkpointer = ModelCheckpoint(filepath='weights.best.from_scratch2.hdf5', verbose=1, save_best_only=True)
print('Start training...')


Start training...


In [12]:
LSTM_model.fit(X_validate1, y_validate1, validation_split = 0.05, 
               epochs=epochs, callbacks=[checkpointer], verbose=1)

Train on 137809 samples, validate on 7254 samples
Epoch 1/10
    96/137809 [..............................] - ETA: 140:05:03 - loss: 0.5558

C:\Users\Shadeh\Anaconda3\envs\Shadehenv\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.124005). Check your callbacks.
  % delta_t_median)


137809/137809 [==============================] - 1195s 9ms/step - loss: 0.5394 - val_loss: 0.5910

Epoch 00001: val_loss improved from inf to 0.59105, saving model to weights.best.from_scratch2.hdf5
Epoch 2/10
137809/137809 [==============================] - 165s 1ms/step - loss: 0.5306 - val_loss: 0.5375

Epoch 00002: val_loss improved from 0.59105 to 0.53751, saving model to weights.best.from_scratch2.hdf5
Epoch 3/10
137809/137809 [==============================] - 182s 1ms/step - loss: 0.5260 - val_loss: 0.5282

Epoch 00003: val_loss improved from 0.53751 to 0.52823, saving model to weights.best.from_scratch2.hdf5
Epoch 4/10
137809/137809 [==============================] - 176s 1ms/step - loss: 0.5218 - val_loss: 0.5000

Epoch 00004: val_loss improved from 0.52823 to 0.50000, saving model to weights.best.from_scratch2.hdf5
Epoch 5/10
137809/137809 [==============================] - 181s 1ms/step - loss: 0.5182 - val_loss: 0.6103

Epoch 00005: val_loss did not improve from 0.50000
Ep

In [13]:
LSTM_model.load_weights('weights.best.from_scratch2.hdf5')

### check model on training data
y_train_pred = LSTM_model.predict(X_train1)
y_train_pred1 = pd.DataFrame(data = y_train_pred, columns = list(y_train.columns))

y_train_pred = y_train_pred1.apply(lambda x: np.exp(x)-1)


for i in list(y_train_pred.columns):
    y_train_pred[i] = y_train_pred[i].apply(lambda x: round(x))

y_train_pred[y_train_pred<0]=0



In [14]:
LSTM_model.load_weights('weights.best.from_scratch2.hdf5')

y_test_pred = LSTM_model.predict(X_test1)

out_cols = list(pd.unique(key_split['date'].values))
y_test_pred1 = pd.DataFrame(data = y_test_pred, columns = list(out_cols))

##Inverse if outlier removed
y_test_pred2 = y_test_pred1.apply(lambda x: np.exp(x)-1)

y_test_pred2[y_test_pred2<0] =0

###merge into Page
test_out = pd.merge(X_test_P, y_test_pred2, left_index = True, right_index = True)
test_out1 = test_out[['Page']+list(test_out.columns.values[-60:])]
test_out_60D = pd.melt(test_out1, id_vars='Page', var_name='date', value_name='Visits_60D')
test_out_60D.head()

,Page,date,Visits_60D
0,2NE1_zh.wikipedia.org_all-access_spider,2017-01-01,22.242765
1,2PM_zh.wikipedia.org_all-access_spider,2017-01-01,25.247850
2,3C_zh.wikipedia.org_all-access_spider,2017-01-01,5.454515
3,4minute_zh.wikipedia.org_all-access_spider,2017-01-01,18.027788
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,2017-01-01,13.765117


In [15]:
results = test_out_120D.merge(test_out_60D)
results['Visits_Seq'] = results[['Visits_120D', 'Visits_60D']].mean(axis =1).apply(lambda x: round(x))

## Merge on 'Page' and 'date'
key_rnn_result = key_split.merge(results, how ='left')
key_rnn_result.head()

MemoryError: 